In [2]:
%matplotlib inline
import pandas as pd
import os
import re
import shutil
import numpy as np
import warnings

In [3]:
df = pd.read_csv('students.csv', sep=';')

In [4]:
df

,family name,name
0,Falkhage,Cornelia
1,Fällman,Amanda
2,Feldt,Axel
3,Flodin,Samuel
4,Florin,Ida
5,Forssgren,Anna
6,Forssmed,Elsa
7,Fredriksson,Mathias
8,Fröberg,Vincent
9,Furborg,Sara


In [5]:
regexp=re.compile(r'^ *(.*)')
def clean_names(name):
    result=regexp.search(name)
    return result.group(1)

In [6]:
df['name']=df['name'].apply(func=clean_names)

In [7]:
def full_name(row):
    return '%s %s' % (row['name'],row['family name'])

In [8]:
df.index = df.apply(func=full_name, axis=1)
assert df.index.is_unique

In [9]:
def create_folder(row,root_path='assignments'):
    
    family_name = row['family name']
    name = row['name']
    dir_name = '%s_%s' % (family_name,name)
    path = os.path.abspath(os.path.join(root_path,dir_name))
    if not os.path.exists(path):
        os.mkdir(path)
    
    return path
    

## Create folders:

In [10]:
root_path='assignments'

if not os.path.exists(root_path):
    os.mkdir(root_path)

df['dir_path'] = df.apply(func=create_folder, axis=1, root_path=root_path)

In [11]:
def make_clickable(val):
    s=r'file:///' + val
    return '<a href="{}">{}</a>'.format(s,"path")

In [12]:
def show(df):
    return df.style.format({'dir_path': make_clickable})

## Sort the assignment files into each student folder
the files from Canvas look something like: *falkhagecornelia_63854_1658103_student_AutomaticGearbox.m*

In [13]:
df['code name'] = df.apply(lambda x:x['family name'].lower()+x['name'].lower(), axis=1)

In [14]:
file_name = 'falkhagecornelia_63854_1658103_student_AutomaticGearbox.m'
regexp = re.compile(pattern=r'([^_]+)_\d+_\d+_(.+)')
regexp.search(file_name).group(1)

'falkhagecornelia'

In [15]:
regexp.search(file_name).group(2)

'student_AutomaticGearbox.m'

In [16]:
submission_dir = 'submissions/'
files = os.listdir(submission_dir)
df_files = pd.DataFrame()

for file_name in files:
    s = pd.Series(dtype='object')
    result = regexp.search(file_name)
    s['code name'] = result.group(1)
    s['file name'] = result.group(2)
    s['original file name'] = file_name
    df_files = df_files.append(s, ignore_index=True)


In [17]:
df_files.head()

,code name,file name,original file name
0,adaanmohamed,Inlämnningsuppgift2.pdf,adaanmohamed_50348_1679286_Inlämnningsuppgift2...
1,adaanmohamed,kaffeOchTe.m,adaanmohamed_50348_1679287_kaffeOchTe.m
2,ahagenalbin,kaffeOchTe.m,ahagenalbin_61667_1679637_kaffeOchTe.m
3,ahagenalbin,kaffeOchTe.pdf,ahagenalbin_61667_1679638_kaffeOchTe.pdf
4,albertssontim,Graf-Kombinerad.pdf,albertssontim_60948_1672688_Graf-Kombinerad.pdf


In [18]:
df_files=pd.merge(left=df, right=df_files, how='outer', left_on='code name', right_on='code name').dropna(subset=['family name'])

In [19]:
df_files.tail()

,family name,name,dir_path,code name,file name,original file name
42,Hansson,Fredrik,E:\dev\TME136-Programming-and-algorithms\2021\...,hanssonfredrik,"Inlämningsuppgift 2, Fredrik Hansson-2.pdf",hanssonfredrik_61305_1672231_Inlämningsuppgift...
43,Hässel,Gustav,E:\dev\TME136-Programming-and-algorithms\2021\...,hässelgustav,kaffeochTe.m,hässelgustav_57869_1660514_kaffeochTe.m
44,Hässel,Gustav,E:\dev\TME136-Programming-and-algorithms\2021\...,hässelgustav,Kaffete.pdf,hässelgustav_57869_1660515_Kaffete.pdf
45,Helland,Wilmer,E:\dev\TME136-Programming-and-algorithms\2021\...,hellandwilmer,kaffeOchTe.m,hellandwilmer_61398_1675937_kaffeOchTe.m
46,Helland,Wilmer,E:\dev\TME136-Programming-and-algorithms\2021\...,hellandwilmer,KaffeTe.pdf,hellandwilmer_61398_1675938_KaffeTe.pdf


### copy the files for each student

In [30]:
def move_assignment_file_to_student_folder_with_correct_file_name(row, submission_dir, additional_files):
    
    if pd.isnull(row['original file name']):
        return
    
    src=os.path.join(submission_dir, row['original file name'])
    dst=os.path.join(row['dir_path'],row['file name'])
    
    if not os.path.exists(dst):
        shutil.copyfile(src,dst)
        
    for additional_file in additional_files:
        
        src=additional_file
        dst=os.path.join(row['dir_path'],os.path.split(additional_file)[-1])
        shutil.copyfile(src,dst)
        

In [31]:
additional_files = ['../kaffeTe.txt','../kaffeTeKort.txt']

df_files.apply(move_assignment_file_to_student_folder_with_correct_file_name, submission_dir=submission_dir, additional_files=additional_files, axis=1);